# 패키지 설치하기

In [ ]:
!pip install ratsnlp

# 환경 설정

모델 hyperparameter와 저장 위치 등 설정 정보를 선언

In [13]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

In [14]:
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/content/drive/Othercomputers/내 컴퓨터/Chapter 5. Sentence_Pair_Classification/checkpoint-paircls",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /content/drive/Othercomputers/내 컴퓨터/Chapter 5. Sentence_Pair_Classification/checkpoint-paircls/epoch=1-val_loss=0.82.ckpt


# 모델 불러오기

파인튜닝을 마친 모델 불러오기

In [15]:
import torch
from transformers import BertConfig, BertForSequenceClassification

In [16]:
# 체크포인트 로드
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

# BERT 설정 로드
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = BertForSequenceClassification(pretrained_model_config)      # 모델 초기화
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})     # 체크포인트 주입
model.eval()    # 평가모드

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

토크나이저 준비

In [17]:
from transformers import BertTokenizer

In [18]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 인퍼런스 함수

In [19]:
def inference_fn(premise, hypothesis):

    # 입력 문장을 토큰화
    inputs = tokenizer(
        [(premise, hypothesis)],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )

    # 평가 모드로 Gradient 업데이트 안함
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})  # 토큰화 된 입력 문장을 텐서로 변환 후 모델에 입력
        prob = outputs.logits.softmax(dim=1)

        # 참, 거짓, 중립 확률을 소수점 두자리에서 반올림
        entailment_prob = round(prob[0][0].item(), 2)
        contradiction_prob = round(prob[0][1].item(), 2)
        neutral_prob = round(prob[0][2].item(), 2)

        # 예측 확률의 최대값 위치에 따른 판단 (0:참, 1:거짓, 2:중립)
        if torch.argmax(prob) == 0:
            pred = "참 (entailment)"
        elif torch.argmax(prob) == 1:
            pred = "거짓 (contradiction)"
        else:
            pred = "중립 (neutral)"

    return {
        'premise': premise,
        'hypothesis': hypothesis,
        'prediction': pred,
        'entailment_data': f"참 {entailment_prob}",
        'contradiction_data': f"거짓 {contradiction_prob}",
        'neutral_data': f"중립 {neutral_prob}",
        'entailment_width': f"{entailment_prob * 100}%",
        'contradiction_width': f"{contradiction_prob * 100}%",
        'neutral_width': f"{neutral_prob * 100}%",
    }

# 웹서비스 개시

`ngrok` 은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구이다. 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 입력해야 한다.

In [20]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

mkdir: cannot create directory ‘/root/.ngrok2’: File exists


인퍼런스 함수를 Flask를 통해 웹서비스로 만든다.

In [21]:
from ratsnlp.nlpbook.paircls import get_web_service_app

In [22]:
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.paircls.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3b7b-35-194-33-249.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Apr/2022 08:37:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2022 08:37:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2022 08:37:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2022 08:38:49] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2022 08:39:24] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2022 08:40:04] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2022 08:40:45] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2022 08:43:00] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2022 08:43:38] "POST /api HTTP/1.1" 200 -
